In [1]:
from get_feature import get_client_cred, get_song_id, get_audio_features
from threading import Thread
import pandas as pd
import spotipy
from requests.exceptions import ConnectionError
import requests
import time

In [2]:
thing = [[] for i in range(8)]
logg = [0 for i in range(8)]
error_count = 0

def write_log():
    print(sum(logg))

In [3]:
def extract_feature(x, i):

    global error_count
    cred = get_client_cred()
    song_artist = x['Artist']
    song_title = x['Title']
    song_artist = song_artist.split("Featuring")[0]
    song_artist = song_artist.split("&")[0]
    obj = None
    try:
        song_id = get_song_id(song_artist, song_title, cred)
        obj = get_audio_features(song_id, cred)
    except requests.exceptions.RequestException as e:    # This is the correct syntax
        error_count += 1
        pass

    if obj:
        x['duration_ms'] = obj['duration_ms']
        x['key'] = obj['key']
        x['acousticness'] = obj['acousticness']
        x['danceability'] = obj['danceability']
        x['energy'] = obj['energy']
        x['instrumentalness'] = obj['instrumentalness']
        x['liveness'] = obj['liveness']
        x['loudness'] = obj['loudness']
        x['speechiness'] = obj['speechiness']
        x['tempo'] = obj['tempo']
        x['valence'] = obj['valence']
        thing[i].append(x)

In [4]:
def feat_util(df, i):
    for y,x in df.iterrows():
        extract_feature(x, i)
        logg[i]+=1

In [5]:
def multi_thread(df):
    print("{} to be done" .format(len(df)))
    arr = [df[i::8] for i in range(8)]
    c = len(arr[7])
    threads = []
    j = 0
    for i in range(8):
        p = Thread(target = feat_util, args = (arr[i], i))
        p.start()
        threads.append(p)

    while(logg[7] < c):
        q = Thread(target = write_log, args = ())
        q.start()
        time.sleep(5)
        q.join()
        
    for i in threads:
        i.join()
    
    print("Error Count: {}".format(error_count))

In [6]:
df = pd.read_csv("billboard_top_100.csv")
df2 = pd.DataFrame(columns = ['Title'])
multi_thread(df)
for i in range(8):
    for j in thing[i]:
        df2 = df2.append(j)
df2.to_csv("pref.csv")

6857 to be done
0
33
52
68
82
102
116
131
149
163
175
188
205
209
212
220
238
262
279
289
295
297
314
346
368
395
424
444
473
501
514
546
573
580
589
597
612
631
643
649
655
670
675
687
700
706
729
755
783
804
827
851
877
917
944
968
1005
1038
1049
1054
1056
1059
1064
1075
1082
1088
1092
1100
1111
1124
1136
1142
1147
1155
1171
1185
1197
1202
1206
1217
1228
1236
1242
1254
1283
1311
1343
1364
1381
1407
1434
1461
1484
1500
1511
1516
1528
1541
1561
1566
1572
1589
1605
1628
1647
1666
1672
1682
1692
1700
1714
1724
1739
1765
1800
1823
1854
1872
1893
1919
1945
1963
1984
1995
2007
2023
2041
2053
2061
2074
2086
2093
2105
2120
2134
2140
2153
2165
2182
2184
2188
2204
2224
2251
2281
2309
2333
2357
2381
2394
2404
2418
2442
2461
2462
2467
2467
2484
2505
2515
2520
2524
2534
2552
2567
2582
2596
2606
2619
2630
2640
2660
2674
2700
2721
2744
2763
2788
2820
2852
2871
2902
2921
2932
2941
2950
2973
2999
3011
3022
3035
3042
3048
3066
3085
3095
3104
3111
3117
3132
3148
3168
3190
3212
3245
3278
3306
3335
3363
3